In [1]:
# LabOne Q:
from laboneq.simple import *

# plotting and fitting functionality
from laboneq.analysis.fitting import (
    lorentzian,
    oscillatory,
    oscillatory_decay,
)
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

# descriptor imports
from laboneq.contrib.example_helpers.descriptors.shfsg_shfqa_pqsc import (
    descriptor_shfsg_shfqa_pqsc,
)

from laboneq.contrib.example_helpers.descriptors.shfsg_shfqa_shfqc_hdawg_pqsc import (
    descriptor_shfsg_shfqa_shfqc_hdawg_pqsc,
)

from laboneq.contrib.example_helpers.randomized_benchmarking_helper import (
    make_pauli_gate_map,
    clifford_parametrized,
    generate_play_rb_pulses,
)

from laboneq.contrib.example_helpers.feedback_helper import (
    state_emulation_pulse,
    create_calibration_experiment,
    create_discrimination_experiment,
)

from laboneq.contrib.example_helpers.generate_example_datastore import (
    generate_example_datastore,
    get_first_named_entry,
)

from zhinst.utils.shfqa.multistate import QuditSettings
from laboneq.analysis import calculate_integration_kernels
# for saving results and pulse sheets
from pathlib import Path
import time

import matplotlib.pyplot as plt
import numpy as np

In [2]:
descriptor = """\
instruments:
  SHFQC:
  - address: DEV12108
    uid: device_shfqc

connections:
  device_shfqc:
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT
    - iq_signal: q0/drive_line_ef
      ports: SGCHANNELS/0/OUTPUT
    - iq_signal: q1/drive_line
      ports: SGCHANNELS/1/OUTPUT
    - iq_signal: q1/drive_line_ef
      ports: SGCHANNELS/1/OUTPUT
    - iq_signal: q2/drive_line
      ports: SGCHANNELS/2/OUTPUT
    - iq_signal: q2/drive_line_ef
      ports: SGCHANNELS/2/OUTPUT
    - iq_signal: q3/drive_line
      ports: SGCHANNELS/3/OUTPUT
    - iq_signal: q3/drive_line_ef
      ports: SGCHANNELS/3/OUTPUT
    - iq_signal: q4/drive_line
      ports: SGCHANNELS/4/OUTPUT
    - iq_signal: q4/drive_line_ef
      ports: SGCHANNELS/4/OUTPUT
    - iq_signal: q5/drive_line
      ports: SGCHANNELS/5/OUTPUT
    - iq_signal: q5/drive_line_ef
      ports: SGCHANNELS/5/OUTPUT
    - iq_signal: q0/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q0/acquire_line
      ports: [QACHANNELS/0/INPUT]
    - iq_signal: q1/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q1/acquire_line
      ports: [QACHANNELS/0/INPUT]
    - iq_signal: q2/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q2/acquire_line
      ports: [QACHANNELS/0/INPUT]
    - iq_signal: q3/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q3/acquire_line
      ports: [QACHANNELS/0/INPUT]
    - iq_signal: q4/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q4/acquire_line
      ports: [QACHANNELS/0/INPUT]
    - iq_signal: q5/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q5/acquire_line
      ports: [QACHANNELS/0/INPUT]
"""

# Define and Load our Device Setup
device_setup = DeviceSetup.from_descriptor(
    yaml_text=descriptor, #descriptor_shfsg_shfqa_shfqc_hdawg_pqsc,
    server_host="10.42.11.0",  # ip address of the LabOne dataserver used to communicate with the instruments
    server_port="8004",  # port number of the dataserver - default is 8004
    setup_name="my_setup",  # setup name
)

def lsg(qubit_str, line_str):
    return device_setup.logical_signal_groups[qubit_str].logical_signals[line_str]

lsg_keys = device_setup.logical_signal_groups.keys()

# perform experiments in emulation mode only? - if True, also generate dummy data for fitting
emulate = False

# create and connect to a session
session = Session(device_setup=device_setup)
session.connect(do_emulation=emulate)

[2024.01.23 23:51:53.040] INFO    Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\chunyans\Documents\Jupyter_notebook\labone_q\laboneq_output\log
[2024.01.23 23:51:53.042] INFO    VERSION: laboneq 2.22.0


[2024.01.23 23:51:53.043] INFO    Connecting to data server at 10.42.11.0:8004
[2024.01.23 23:51:53.336] INFO    Connected to Zurich Instruments LabOne Data Server version 23.10 at 10.42.11.0:8004
[2024.01.23 23:51:53.494] WARNING SHFQC/QA:dev12108: Include the device options 'SHFQC/16W/QC6CH/RTR' in the device setup ('options' field of the 'instruments' list in the device setup descriptor). This will become a strict requirement in the future.
[2024.01.23 23:51:57.559] INFO    Configuring the device setup
[2024.01.23 23:51:57.668] INFO    The device setup is configured


In [3]:

num_qudits = len(lsg_keys)

ro_pulse_duration = 1e-6

qudits_params = {
"qudits":lsg_keys,
"ro_num_states": num_qudits * [2],
"ro_cent_f": 0,
"ro_df":     num_qudits * [10e6],
"ro_pin_range":   0,
"ro_pout_range":  0,
"ro_pulse_data": {},
"ro_pulse_delay": 0,
"ro_int_t": ro_pulse_duration,
"ro_int_delay": 0e-9,

"ro_int_weights": {},
"thresholds":{},
"ro_snr": num_qudits * [0],
"ro_fidelity": num_qudits * [0],
"ro_fidelity_ef": num_qudits * [0],
"ro_x_talk_matrix": {},
"wait_after_int": 5e-6,

"dr_cent_f": num_qudits * [7e9], # <= 3 different LO f, because ch1 and ch2 share the same LO, and same for ch3 and ch4, ch5 and ch6
"dr_df": num_qudits * [10e6],
"dr_df_ef": num_qudits * [10e6],
"dr_p_range":   num_qudits * [0],
"dr_pulse_data": num_qudits * [0.5],
"dr_pulse_data_ef": num_qudits * [0.5],
"dr_pi_pulse": {},
"dr_pi_pulse_ef": {},
"dr_pi/2_pulse": {},
"dr_pi/2_pulse_ef": {},
"T1": num_qudits * [0],
"T2": num_qudits * [0],
"T2_echo": num_qudits * [0],
"T2_Ramsey": num_qudits * [0],
"T2_CPMG": num_qudits * [0],
"T1_ef": num_qudits * [0],
"T2_echo_ef": num_qudits * [0],
"T2_Ramsey_ef": num_qudits * [0],
"T2_CPMG_ef": num_qudits * [0],
"1q_gate_fidelity": num_qudits * [0],
"2q_gate_fidelity": num_qudits * [0],
"dr_x_talk_matrix": {},
}

states_str = ["g", "e", "f", "h"]

In [4]:
from pathlib import Path
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import scipy as scy
from scipy.signal import gaussian
from scipy.optimize import leastsq
from scipy.stats import norm
from pylab import rcParams
import textwrap
import itertools
# function that defines a resonator spectroscopy experiment, and takes the frequency sweep as a parameter


ro_port_mode = PortMode.LF if qudits_params["ro_cent_f"] == 0 else PortMode.RF
print(ro_port_mode)
def define_calibration(qudits_params):
    # the calibration object will later be applied to the device setup
    calibration = Calibration()

    for qn in qudits_params["qudits"]:
        calibration[lsg(qn, "acquire_line")] = SignalCalibration(
            local_oscillator = Oscillator(frequency = qudits_params["ro_cent_f"]),
            oscillator =       Oscillator(frequency = qudits_params["ro_df"][int(qn[1])],
                                    modulation_type = ModulationType.SOFTWARE),
            # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse
            port_delay = qudits_params["ro_pulse_delay"] + qudits_params["ro_int_delay"],
            range = qudits_params["ro_pin_range"],
            port_mode=ro_port_mode,

        )
        # measure line
        calibration[lsg(qn, "measure_line")] = SignalCalibration(
            local_oscillator = Oscillator(frequency = qudits_params["ro_cent_f"]),
            oscillator =       Oscillator(frequency = qudits_params["ro_df"][int(qn[1])],
                                modulation_type = ModulationType.SOFTWARE),
            port_delay = qudits_params["ro_pulse_delay"] ,
            range = qudits_params["ro_pout_range"],
            port_mode=ro_port_mode,
        )
        calibration[lsg(qn, "drive_line")] = SignalCalibration(
            # verify which qubits share the same LOs!
            local_oscillator = Oscillator(frequency = qudits_params["dr_cent_f"][int(qn[1])]),
            # each logical signal can have an oscillator associated with it
            oscillator =       Oscillator(frequency = qudits_params["dr_df"][int(qn[1])],
                                modulation_type = ModulationType.HARDWARE),
            range = qudits_params["dr_p_range"][int(qn[1])],
        )
        calibration[lsg(qn, "drive_line_ef")] = SignalCalibration(
        # verify which qubits share the same LOs!
        local_oscillator = Oscillator(frequency = qudits_params["dr_cent_f"][int(qn[1])]),
        # each logical signal can have an oscillator associated with it
        oscillator =       Oscillator(frequency = qudits_params["dr_df_ef"][int(qn[1])],
                            modulation_type = ModulationType.HARDWARE),
        range = qudits_params["dr_p_range"][int(qn[1])],
        )
    return calibration

calibration = define_calibration(qudits_params)
device_setup.set_calibration(calibration)
g1_len = 10e-6
g2_len = 2e-6

g1_pulse = pulse_library.const(uid = "g1_pulse", amplitude = 0.3, length = g1_len, can_compress=True)
g2_pulse = pulse_library.const(uid = "g2_pulse", amplitude = 0.3, length = g2_len, can_compress=True)

amp_sweep_g1 = LinearSweepParameter("amp_g1", start=0.1, stop = 0.3, count=3)

def charge_stability_diagram():
    exp = Experiment(uid="exp", 
        signals = [
                ExperimentSignal(("drive_g1"),   map_to=lsg("q0", "drive_line")), 
                ExperimentSignal(("drive_g2"),   map_to=lsg("q1", "drive_line")),                   
                ExperimentSignal(("measure_g"), map_to=lsg("q0", "measure_line")),
                ExperimentSignal(("acquire_g"), map_to=lsg("q0", "acquire_line")),  
        ]  
    )
    with exp.acquire_loop_rt(
        uid="exp",
        count=2**0,
        acquisition_type = AcquisitionType.SPECTROSCOPY,
        averaging_mode = AveragingMode.CYCLIC,
    ):
        with exp.sweep(uid='sweep amp g1', parameter=amp_sweep_g1):
            with exp.section(uid="g1_sweep"):
                with exp.section(uid = "g1_1"):
                    exp.play(signal = "drive_g1", pulse=g1_pulse, amplitude = amp_sweep_g1)

                for i in range(int(g1_len/g2_len)):
                    with exp.section(uid= f"g2_sweep_{i}", alignment=SectionAlignment.RIGHT):
                        exp.play(signal = "drive_g2", pulse=g2_pulse, amplitude = 0.1+0.1*i)
                        exp.acquire(
                            signal="acquire_g",
                            handle="results",
                            length=1e-6,
                        )

    cal = Calibration()
  
    cal["drive_g1"] = SignalCalibration(
        local_oscillator=Oscillator(frequency=5e9),
        range=0)
    cal["drive_g2"] = SignalCalibration(
        local_oscillator=Oscillator(frequency=5e9),
        range=0)
    exp.set_calibration(cal)

               
    compiler_settings = None
    compiled_exp = session.compile(exp, compiler_settings = compiler_settings)
    results = session.run(compiled_exp)    
    Path("Pulse_Sheets").mkdir(parents=True, exist_ok=True)
    show_pulse_sheet(f'''Pulse_Sheets/csd''', compiled_exp) 

    return results

results = charge_stability_diagram()

data = results.acquired_results["results"].data

PortMode.LF
[2024.01.23 23:51:57.992] INFO    Starting LabOne Q Compiler run...
[2024.01.23 23:51:57.997] INFO    Schedule completed
[2024.01.23 23:51:58.051] INFO     ───────────────────────────────────────────────────────────────────── 
[2024.01.23 23:51:58.051] INFO      Device            AWG   SeqC LOC   CT entries   Waveforms   Samples  
[2024.01.23 23:51:58.052] INFO     ───────────────────────────────────────────────────────────────────── 
[2024.01.23 23:51:58.052] INFO      device_shfqc        0         14            0           0         0  
[2024.01.23 23:51:58.053] INFO      device_shfqc_sg     0         12            3           1        64  
[2024.01.23 23:51:58.054] INFO      device_shfqc_sg     1         18            5           1        64  
[2024.01.23 23:51:58.054] INFO     ───────────────────────────────────────────────────────────────────── 
[2024.01.23 23:51:58.055] INFO      TOTAL                         44            8                   128  
[2024.01.23 23:51:5